In [1]:
from scrollguard.utils import get_config, get_mongo_client
from scrollguard import etl
import numpy as np
import requests
import ssl
import urllib3
import xmltodict
import pandas as pd

pd.set_option("display.max_column", None)

In [2]:
""" BEGIN SECTION
ISSUE: SSLError when downloading UN source via requests
ERROR: SSLError: HTTPSConnectionPool(host='scsanctions.un.org', port=443): 
    Max retries exceeded with url: /resources/xml/en/consolidated.xml 
    (Caused by SSLError(SSLError(1, '[SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED] 
    unsafe legacy renegotiation disabled (_ssl.c:1007)')))
DESCRIPTION: Encounters this issue on WSL2 Ubuntu 22.04.3 LTS (jammy), 
    but not on regular Windows machine.
SOURCE: https://stackoverflow.com/a/73519818
"""
class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.
    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)

def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount("https://", CustomHttpAdapter(ctx))
    return session

""" END SECTION
"""

' END SECTION\n'

In [3]:
raw = etl.extract_collection("UN_INDIVIDUAL_RAW")
processed = etl.Transformer("UN_INDIVIDUAL",True)
processed.processed_data

,SOURCE_NAME,ID,FULL_NAME,NAME_TYPE,DATE_OF_BIRTH,COUNTRY_OF_BIRTH,NATIONALITY,STANDARDIZED_NAME,METAPHONE_NAME
0,UN_INDIVIDUAL,110447,MUHAMMAD TAHER ANWARI,INDIVIDUAL,NaN,Afghanistan,Afghanistan,MUHAMMAD TAHER ANWARI,MHMT THR ANWR
1,UN_INDIVIDUAL,110554,ABDUL LATIF MANSUR,INDIVIDUAL,NaN,Afghanistan,Afghanistan,ABDUL LATIF MANSUR,ABTL LTF MNSR
2,UN_INDIVIDUAL,110577,MOHAMMAD HASSAN AKHUND,INDIVIDUAL,NaN,Afghanistan,Afghanistan,MOHAMMAD HASSAN AKHUND,MHMT HSN AKHNT
3,UN_INDIVIDUAL,110578,ABDUL KABIR MOHAMMAD JAN,INDIVIDUAL,NaN,Afghanistan,Afghanistan,ABDUL KABIR MOHAMMAD JAN,ABTL KBR MHMT JN
4,UN_INDIVIDUAL,110579,MOHAMMED OMAR GHULAM NABI,INDIVIDUAL,NaN,Afghanistan,Afghanistan,MOHAMMED OMAR GHULAM NABI,MHMT OMR KHLM NB
...,...,...,...,...,...,...,...,...,...
703,UN_INDIVIDUAL,6908843,NOOR WALI MEHSUD,INDIVIDUAL,1978-06-26,Pakistan,Pakistan,NOOR WALI MEHSUD,NR WL MST
704,UN_INDIVIDUAL,2983834,HAFIZ ABDUL SALAM BHUTTAVI,INDIVIDUAL,NaN,Pakistan,Pakistan,HAFIZ ABDUL SALAM BHUTTAVI,HFS ABTL SLM BHTF
705,UN_INDIVIDUAL,2901762,TAYEB NAIL,INDIVIDUAL,NaN,Algeria,Algeria,TAYEB NAIL,TYB NL
706,UN_INDIVIDUAL,112037,KAMEL DJERMANE,INDIVIDUAL,1965-10-12,Algeria,Algeria,KAMEL DJERMANE,KML TJRMN
